<a href="https://colab.research.google.com/github/sazio/first-order-model/blob/master/first_order_model_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [1]:
#!git clone https://github.com/AliaksandrSiarohin/first-order-model
!git clone https://github.com/sazio/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 249 (delta 1), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (249/249), 72.19 MiB | 31.16 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [2]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [4]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")


#source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
source_image = imageio.imread("/content/gdrive/My Drive/first-order-motion-model/camma.png")
#driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/plomo.mp4')

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [6]:
!pip install mhmovie

In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                          checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')#, cpu = True)

Audio Extraction with FFMPEG

In [34]:
path_to_vid = '/content/gdrive/My Drive/first-order-motion-model/plomo.mp4'
path_to_audio = path_to_vid[:50] +  path_to_vid[50:-4] + '-audio'+ '.wav'

In [24]:
import os, sys, re

video_file_path =  path_to_vid#@param 
output_file_extension = 'wav' #@param ["m4a", "mp3", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

!ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/gdrive/My Drive/first-order-motion-model/plomo.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.28.101
  Duration: 00:00:04.07, start: 0.000000, bitrate: 778 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 528x294 [SAR 1:1 DAR 88:49], 672 kb/s, 29.97 fps, 29.97 tbr, 30k tbn, 59.94 tbc (default)
    Metadata:
      handler_name    : ISO Media file produced by Google Inc. Created on: 11/06/2018.
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 96 kb/s (default)
    Metadata:
      handler_name    : ISO Media file produced by Google Inc. Created on: 11/06/2018.
File '/content/gdrive/My Drive/first-order-motion-model//plomo-audio.wav' already exists. Overwrite ? [y/N] y
Output #0, wav, to '/content/gdrive/My Drive/first-order-motion-model//plomo-audio.wav':
  Metadata:
    major_brand     :

**Perform image animation**

In [46]:
from demo import make_animation
from skimage import img_as_ubyte
from mhmovie import code

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
path_gen_vid = '/content/gdrive/My Drive/first-order-motion-model/generated.mp4'
imageio.mimsave(path_gen_vid, [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

#HTML(display(source_image, driving_video, predictions).to_html5_video())
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 122/122 [00:04<00:00, 25.07it/s]


In [49]:
path_final_vid = '/content/gdrive/My Drive/first-order-motion-model/final_vid.mp4'
os.environ["inputVideo"] = path_gen_vid
os.environ["inputAudio"] = path_to_audio
os.environ["outputVideo"] = path_final_vid

!ffmpeg -hide_banner -i "$inputVideo" -i "$inputAudio" -c:v copy  -c:a aac "$outputVideo"

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/gdrive/My Drive/first-order-motion-model/generated.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
  Duration: 00:00:12.20, start: 0.000000, bitrate: 64 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 256x256, 63 kb/s, 10 fps, 10 tbr, 10240 tbn, 20 tbc (default)
    Metadata:
      handler_name    : VideoHandler
[aac @ 0x564133882800] Multiple frames in a packet.
Input #1, wav, from '/content/gdrive/My Drive/first-order-motion-model/plomo-audio.wav':
  Metadata:
    encoder         : Lavf57.83.100
  Duration: 00:00:04.06, bitrate: 96 kb/s
    Stream #1:0: Audio: aac (LC) ([255][0][0][0] / 0x00FF), 44100 Hz, stereo, fltp, 39 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (copy)
  Stream #1:0 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[aac @ 0x564133882300] Multiple frames in a packet.
[aac

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 122/122 [00:04<00:00, 25.85it/s]


## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [ ]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/09.png')
driving_video = imageio.mimread('hinton.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 240/240 [00:08<00:00, 29.00it/s]
